In [ ]:
from nwm_reanalysis import NWM
from dask.distributed import Client, LocalCluster
# import asyncio

In [ ]:
start_date = "2000-01-01"
end_date = "2010-12-31"
# comids = [2043493, 20873280, 7590453]
comids = [2043493, 20873280, 7590453]


nwm = NWM(start_date=start_date, end_date=end_date, comids=comids)
scheduler = LocalCluster(n_workers=10, threads_per_worker=2, processes=False)
scheduler

In [ ]:
%%time
# %%prun
nwm.request_timeseries(scheduler=scheduler, optimize=True, nwm_21=True)
nwm.data
# 10y, 1comid: 14min CPU, 19min
# 10y, 2comid: 21min CPU, 30min
# 10y, 3comid: 32min CPU, 46min

In [ ]:
%%time
df = nwm.set_output(return_dataframe=False)
nwm.output

In [ ]:
%%time
nwm.request_timeseries(scheduler=scheduler, optimize=False, nwm_2=True)
nwm.data

In [ ]:
%%time
nwm.set_output()
nwm.output


-----------------------------------

In [53]:
import xarray as xr
import fsspec
import dask
import s3fs
from datetime import datetime
from rechunker import rechunk

from dask.distributed import Client, LocalCluster

In [54]:
n_workers = 10
scheduler = LocalCluster(n_workers=n_workers, threads_per_worker=2, processes=False)
client = Client(scheduler)
client.dashboard_link

C:\Users\dsmith\Anaconda3\envs\hms_kube\lib\site-packages\distributed\node.py:181: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 53381 instead
  warnings.warn(


'http://10.104.12.219:53381/status'

In [55]:
# nwm_url = 's3://noaa-nwm-retrospective-2-1-zarr-pds/chrtout.zarr'
nwm_url = 's3://noaa-nwm-retrospective-2-1-zarr-pds/lakeout.zarr'
start_date = datetime.fromisoformat("2000-01-01")
end_date = datetime.fromisoformat("2010-12-31")

# STREAMFLOW 
# comids = [2043493, 20873280, 7590453]
# comids = [2043493]
# variables = ["streamflow", "velocity"]

# LAKE
comids = [166757656]
# comids = [717696]
variables = ["inflow", "outflow", "water_sfc_elev"]

start_string = f"{start_date.year}-{start_date.month}-{start_date.day}"
end_string = f"{end_date.year}-{end_date.month}-{end_date.day}"

In [56]:
%%time
# drop_variables = ["elevation", "order", "qBtmVertRunoff", "qBucket", "qSfcLatRunoff", "q_lateral"]
ds = xr.open_zarr(fsspec.get_mapper(nwm_url, anon=True), consolidated=True)
ds

CPU times: total: 1.67 s
Wall time: 5.33 s


<xarray.Dataset>
Dimensions:         (feature_id: 5783, time: 367439)
Coordinates:
  * feature_id      (feature_id) int32 491 531 747 ... 947070204 1021092845
    latitude        (feature_id) float32 dask.array<chunksize=(5783,), meta=np.ndarray>
    longitude       (feature_id) float32 dask.array<chunksize=(5783,), meta=np.ndarray>
  * time            (time) datetime64[ns] 1979-02-01T01:00:00 ... 2020-12-31T...
Data variables:
    crs             |S1 ...
    inflow          (time, feature_id) float64 dask.array<chunksize=(8064, 500), meta=np.ndarray>
    outflow         (time, feature_id) float64 dask.array<chunksize=(8064, 500), meta=np.ndarray>
    water_sfc_elev  (time, feature_id) float32 dask.array<chunksize=(8064, 500), meta=np.ndarray>
Attributes:
    Conventions:                  CF-1.6
    TITLE:                        OUTPUT FROM WRF-Hydro v5.2.0-beta2
    code_version:                 v5.2.0-beta2
    featureType:                  timeSeries
    model_configuration:          retrospective
    model_output_type:            reservoir
    proj4:                        +proj=lcc +units=m +a=6370000.0 +b=6370000....
    reservoir_assimilated_value:  Assimilation not performed
    reservoir_type:               1 = level pool everywhere
    station_dimension:            lake_id

In [57]:
%%time
# Option one, default with comid selection and time slice happening at the same time with dask
with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ds_data = ds[variables].sel(feature_id=comids).sel(time=slice(start_string, end_string)).load(optimize_graph=False, traverse=False)
data = ds_data
print(f"Start: {start_string}, End: {end_string}")
data = data.to_dataframe()
data

Start: 2000-1-1, End: 2010-12-31
CPU times: total: 23.4 s
Wall time: 37.9 s


,,inflow,outflow,water_sfc_elev,latitude,longitude
time,feature_id,,,,,
2000-01-01 00:00:00,166757656,50.449999,82.409998,9.965346,26.945404,-80.833214
2000-01-01 01:00:00,166757656,50.419999,82.409998,9.965278,26.945404,-80.833214
2000-01-01 02:00:00,166757656,50.399999,82.399998,9.965209,26.945404,-80.833214
2000-01-01 03:00:00,166757656,50.379999,82.389998,9.965140,26.945404,-80.833214
2000-01-01 04:00:00,166757656,50.349999,82.389998,9.965072,26.945404,-80.833214
...,...,...,...,...,...,...
2010-12-31 19:00:00,166757656,26.129999,59.529999,9.646295,26.945404,-80.833214
2010-12-31 20:00:00,166757656,26.119999,59.529999,9.646226,26.945404,-80.833214
2010-12-31 21:00:00,166757656,26.109999,59.519999,9.646157,26.945404,-80.833214


In [58]:
# calculate volume using surface area from nhdplus dataset and the lakeparm.nc file
import sqlite3

catchments_path = "D:\\git\\hms_kube\\hms\\Web.Services\\App_Data\\catchments.sqlite"
lakeparm_path = "D:\\data\\hms\\LAKEPARM_CONUS.nwm.v2.1.nc"

# Retrieve area from db
# con = sqlite3.connect(catchments_path)
# cur = con.cursor()
# surfacearea = cur.execute(f"SELECT AREASQKM FROM Waterbodies WHERE COMID={comids[0]}").fetchall()[0][0]
# con.close()

from netCDF4 import Dataset
import xarray as xr
import numpy as np
# lakenc = Dataset(lakeparm_path)
lakenc = xr.open_dataset(lakeparm_path)
lakenc

<xarray.Dataset>
Dimensions:         (feature_id: 5783)
Coordinates:
    lat             (feature_id) float32 ...
    lon             (feature_id) float32 ...
Dimensions without coordinates: feature_id
Data variables: (12/14)
    Dam_Length      (feature_id) int32 ...
    LkMxE           (feature_id) float64 ...
    OrificeE        (feature_id) float64 ...
    WeirE           (feature_id) float64 ...
    lake_id         (feature_id) int32 ...
    LkArea          (feature_id) float64 ...
    ...              ...
    OrificeC        (feature_id) float64 ...
    OrificeA        (feature_id) float64 ...
    time            (feature_id) datetime64[ns] ...
    ascendingIndex  (feature_id) int32 ...
    ifd             (feature_id) float32 ...
    crs             |S1 ...
Attributes:
    Conventions:  CF-1.5
    featureType:  timeSeries
    history:      Fri Sep  6 09:47:15 2019: ncatted -O -a units,Dam_Length,c,...
    NCO:          netCDF Operators version 4.7.9 (Homepage = http://nco.sf.ne...
    region:       CONUS
    version:      NWM v2.1

In [64]:
# Convert xarray dataset to a pandas dataframe and select row where lake_id == comid

lake_df = lakenc[["lake_id", "LkArea", "OrificeE"]].to_dataframe()
lake_data = lake_df.loc[lake_df['lake_id'].isin(comids)]
lake_data

,lake_id,LkArea,OrificeE,lat,lon
feature_id,,,,,
423,166757656,1726.619995,4.82,26.945404,-80.833214


distributed.comm.inproc - WARNING - Closing dangling queue in <InProc  local=inproc://10.104.12.219/20092/1 remote=inproc://10.104.12.219/20092/165>


In [60]:
# Calculate volume: (LkArea*1000000) * (water_surface_elevation - OrificeE)
lake_area = float(lake_data["LkArea"])
orifice_e = float(lake_data["OrificeE"])
print(f"Lake Area: {lake_area}, OrificeE: {orifice_e}")

Lake Area: 1726.6199951171875, OrificeE: 4.820000012715657


In [61]:
data["volume"] = (lake_area*1000000) * (data["water_sfc_elev"] - orifice_e)
data

,,inflow,outflow,water_sfc_elev,latitude,longitude,volume
time,feature_id,,,,,,
2000-01-01 00:00:00,166757656,50.449999,82.409998,9.965346,26.945404,-80.833214,8.884058e+09
2000-01-01 01:00:00,166757656,50.419999,82.409998,9.965278,26.945404,-80.833214,8.883939e+09
2000-01-01 02:00:00,166757656,50.399999,82.399998,9.965209,26.945404,-80.833214,8.883821e+09
2000-01-01 03:00:00,166757656,50.379999,82.389998,9.965140,26.945404,-80.833214,8.883702e+09
2000-01-01 04:00:00,166757656,50.349999,82.389998,9.965072,26.945404,-80.833214,8.883584e+09
...,...,...,...,...,...,...,...
2010-12-31 19:00:00,166757656,26.129999,59.529999,9.646295,26.945404,-80.833214,8.333177e+09
2010-12-31 20:00:00,166757656,26.119999,59.529999,9.646226,26.945404,-80.833214,8.333058e+09
2010-12-31 21:00:00,166757656,26.109999,59.519999,9.646157,26.945404,-80.833214,8.332939e+09


In [27]:
%%time
ds = ds.sel(time=slice(start_string, end_string))
ds_features_0 = (ds.where(ds.feature_id.isin(comids), drop=True))
ds_features = ds_features_0.copy()
ds_features = ds_features.drop('crs')

# Option two, rechunk the data
dim_chunk_sizes = {'feature_id': len(comids), 'time': len(ds_features.time)}
ds_features = ds_features.chunk(chunks=dim_chunk_sizes)
chunk_plan = {}
for vv in ds_features.variables:
    if vv in ['streamflow', 'velocity']:
        chunk_plan[vv] = tuple((dim_chunk_sizes[tt] for tt in ds_features[vv].dims))
    else: 
        chunk_plan[vv] = ds_features[vv].shape
    ds_features[vv].encoding['chunks'] = None

max_mem = f"{format(0.9 * 56 / n_workers, '.2f')}GB"
ds_features = ds_features.chunk(chunks=dim_chunk_sizes)
intermediate = "intermediate.zarr"
target = "target.zarr"

rechunked = rechunk(ds_features, target_chunks=chunk_plan, target_store=target,
                    max_mem=max_mem,
                    temp_store=intermediate)
result = rechunked.execute(retries=10)
ds = xr.open_zarr(target)
ds_data = ds[variables].load(optimize_graph=True, traverse=False)
data = ds_data
print(f"Start: {start_string}, End: {end_string}")
data

NameError: name 'ds' is not defined

In [ ]:
%%time
# Option three, no dask slicing
ds_data = ds[variables].sel(feature_id=comids).sel(time=slice(start_string, end_string)).load(optimize_graph=False, traverse=False)
data = ds_data
print(f"Start: {start_string}, End: {end_string}")
data

In [ ]:
%%time
data = data.sel(time=slice(start_string, end_string))
data

In [ ]:
%%time
# zarr_file = nwm_url
# s3 = s3fs.S3FileSystem(anon=True)
# store = s3fs.S3Map(root=zarr_file, s3=s3, check=False)
# ds_nwm_chrtout = xr.open_zarr(store=store, consolidated=True)
# ds_nwm_chrtout

In [ ]:
%%time
ds_nwm_feature = ds_nwm_chrtout.isel(feature_id==f'{self.comids}'.encode())
streamflow_nwm = ds_nwm_feature.streamflow.load()
streamflow_nwm_df = streamflow_nwm.squeeze('feature_id').to_dataframe()
streamflow_nwm_df

In [ ]:
%%time
variables = ["streamflow", "velocity"]
start_string = f"{nwm.start_date.year}-{nwm.start_date.month}-{nwm.start_date.day}"
end_string = f"{nwm.end_date.year}-{nwm.end_date.month}-{nwm.end_date.day}"
# with dask.config.set(**{'array.slicing.split_large_chunks': True}):
ds_data = ds[variables].sel(feature_id=nwm.comids).sel(time=slice(start_string, end_string)).load(optimize_graph=False, traverse=False)
data = ds_data
print(f"Start: {start_string}, End: {end_string}")
data

In [ ]:
%%time
start_string = f"{nwm.start_date.year}-{nwm.start_date.month}-{nwm.start_date.day}"
end_string = f"{nwm.end_date.year}-{nwm.end_date.month}-{nwm.end_date.day}"
ds_data = ds.sel(feature_id=nwm.comids)
ds_data = ds_data.sel(time=slice(start_string, end_string)).load(optimize_graph=True, traverse=False).load()
ds_data

In [ ]:
ds_data = ds_data.sel(time=slice(start_string, end_string)).load(optimize_graph=True, traverse=False).load()
print(f"Start: {start_string}, End: {end_string}")
ds_data

In [ ]:
all_variables = ["elevation", "order", "qBtmVertRunoff", "qBucket", "qSfcLatRunoff", "q_lateral", "streamflow", "velocity"]
d_v = list(set(all_variables) - set(variables))
d_v